In [14]:
import os
import sys
sys.path.append('..')

from azure.identity import DefaultAzureCredential

from src.financialqa.ingestion.parser import extract_pdf_contents, page_pdf_contents
from src.financialqa.ingestion.ingestionpipeline import IngestionPipeline
from src.financialqa.inference.inferencepipeline import InferencePipeline

from dotenv import load_dotenv
load_dotenv('../.env', override=True)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
token = DefaultAzureCredential().get_token('https://cognitiveservices.azure.com/.default')
os.environ['AZURE_OPENAI_API_KEY'] = token.token

21:05:04 INFO azure.identity._credentials.environment line 115  No environment configuration found.
21:05:04 INFO azure.identity._credentials.managed_identity line 88  ManagedIdentityCredential will use Azure ML managed identity
21:05:04 INFO azure.core.pipeline.policies.http_logging_policy line 509  Request URL: 'http://127.0.0.1:46808/MSI/auth?api-version=REDACTED&resource=REDACTED'
Request method: 'GET'
Request headers:
    'secret': 'REDACTED'
    'User-Agent': 'azsdk-python-identity/1.19.0 Python/3.9.21 (Linux-5.15.0-1078-azure-x86_64-with-glibc2.31)'
No body was attached to the request
21:05:05 INFO azure.core.pipeline.policies.http_logging_policy line 545  Response status: 200
Response headers:
    'Content-Type': 'application/json'
    'Date': 'Fri, 07 Feb 2025 21:05:05 GMT'
    'Transfer-Encoding': 'chunked'
21:05:05 INFO azure.identity._credentials.chained line 124  DefaultAzureCredential acquired a token from ManagedIdentityCredential


In [15]:
ingestion_pipeline = IngestionPipeline()

21:06:57 INFO src.financialqa.ingestion.ingestionpipeline line 87  Configured logging
21:06:57 INFO src.financialqa.ingestion.ingestionpipeline line 793  Loaded API variables
21:06:57 INFO src.financialqa.ingestion.ingestionpipeline line 721  Getting Azure Blob Storage container client 'messiah-test' from account 'eliasstorage2'
21:06:57 INFO src.financialqa.ingestion.ingestionpipeline line 743  Getting Azure AI Search client from service 'nlp-ai-search1'..
21:06:57 INFO src.financialqa.ingestion.ingestionpipeline line 761  Getting OpenAI embedding model 'text-embedding-ada-002' from endpoint https://jhi-apim.prod.manulife.com/internal/technologystrategymgt/usdoltcaoaiprod/v1/usa..


In [4]:
pdfs_folder = os.path.join('../data')

In [5]:
pdf_contents = extract_pdf_contents(pdfs_folder)

21:05:08 INFO src.financialqa.ingestion.parser line 54  Extracting contents from 'CIBC_Preview_Q1'...


In [6]:
pdf_pages_dict = page_pdf_contents(pdf_contents)

In [7]:
print(pdf_pages_dict.get('CIBC_Preview_Q1').get('pages').get(1).keys())
print(pdf_pages_dict.get('CIBC_Preview_Q1').get('pages').get(2).keys())
print(pdf_pages_dict.get('CIBC_Preview_Q1').get('pages').get(3).keys())

dict_keys(['text', 'tables', 'sectionHeading', 'footnote', 'pageFooter', 'figures'])
dict_keys(['text', 'tables', 'title', 'pageFooter', 'pageNumber', 'figures'])
dict_keys(['text', 'tables', 'pageNumber', 'pageFooter'])


In [8]:
ingestion_pipeline.crop_images_from_pdfs(
    pdf_pages_dict,
    save_as_jpg=True,
)

21:05:42 INFO src.financialqa.ingestion.ingestionpipeline line 147  Processing figures on page 1 of 'CIBC_Preview_Q1'...


21:05:42 INFO src.financialqa.ingestion.ingestionpipeline line 196  Saving figure 'figure_1_page_1' as PDF to folder ../data/outputs/CIBC_Preview_Q1/figure_1_page_1.pdf
21:05:43 INFO src.financialqa.ingestion.ingestionpipeline line 217  Saving figure 'figure_1_page_1' as JPEG to folder ../data/outputs/CIBC_Preview_Q1/jpg_outputs
21:05:43 INFO src.financialqa.ingestion.ingestionpipeline line 147  Processing figures on page 2 of 'CIBC_Preview_Q1'...
21:05:43 INFO src.financialqa.ingestion.ingestionpipeline line 196  Saving figure 'figure_1_page_2' as PDF to folder ../data/outputs/CIBC_Preview_Q1/figure_1_page_2.pdf
21:05:43 INFO src.financialqa.ingestion.ingestionpipeline line 217  Saving figure 'figure_1_page_2' as JPEG to folder ../data/outputs/CIBC_Preview_Q1/jpg_outputs
21:05:43 INFO src.financialqa.ingestion.ingestionpipeline line 147  Processing figures on page 5 of 'CIBC_Preview_Q1'...
21:05:43 INFO src.financialqa.ingestion.ingestionpipeline line 196  Saving figure 'figure_1_page

In [8]:
ingestion_pipeline.generate_table_from_chart(
    pdf_pages_dict, 
    chart_model='chartvlm',
    chart_model_path='',
)

19:07:05 INFO src.financialqa.ingestion.ingestionpipeline line 198  Generating ChartVLM output for figure with JPG path: ../data/outputs/CIBC_Preview_Q1/jpg_outputs/figure_1_page_1.jpg...


/home/azureuser/cloudfiles/code/Users/ataeyme/projects/FinancialQA/src/financialqa/ingestion/models/ChartVLM-base/base_decoder


Legacy behavior is being used. The current behavior will be deprecated in version 5.0.0. In the new behavior, If both images and text are provided, image_processor is not a VQA processor, and `add_special_tokens` is unset, the default value of `add_special_tokens` will be changed to `False` when calling the tokenizer. To test the new behavior, set `legacy=False`as a processor call argument.


In [9]:
text_docs, table_docs, chart_docs = ingestion_pipeline.convert_paged_pdf_contents_to_docs(
    pdf_pages_dict,
    convert_tables=True,
    convert_charts=True,
)

21:05:53 INFO src.financialqa.ingestion.ingestionpipeline line 342  Converting extracted 'CIBC_Preview_Q1' contents to Documents...
21:05:53 INFO src.financialqa.ingestion.ingestionpipeline line 405  Missing chart table for figure_1_page_1 of 'CIBC_Preview_Q1'
21:05:53 INFO src.financialqa.ingestion.ingestionpipeline line 405  Missing chart table for figure_1_page_2 of 'CIBC_Preview_Q1'
21:05:53 INFO src.financialqa.ingestion.ingestionpipeline line 405  Missing chart table for figure_1_page_5 of 'CIBC_Preview_Q1'
21:05:53 INFO src.financialqa.ingestion.ingestionpipeline line 405  Missing chart table for figure_1_page_6 of 'CIBC_Preview_Q1'
21:05:53 INFO src.financialqa.ingestion.ingestionpipeline line 405  Missing chart table for figure_1_page_10 of 'CIBC_Preview_Q1'
21:05:53 INFO src.financialqa.ingestion.ingestionpipeline line 423  Created 20 text, 17 table, and 0 chart Documents for PDF CIBC_Preview_Q1


In [10]:
text_doc_chunks = ingestion_pipeline.chunk_docs(text_docs, chunk_size=400)
table_doc_chunks = ingestion_pipeline.chunk_docs(table_docs, chunk_size=400)
chart_doc_chunks = ingestion_pipeline.chunk_docs(chart_docs, chunk_size=400)

21:05:55 INFO src.financialqa.ingestion.ingestionpipeline line 446  Chunked 20 Documents to 36 using a chunk size of 400 
21:05:55 INFO src.financialqa.ingestion.ingestionpipeline line 446  Chunked 17 Documents to 41 using a chunk size of 400 
21:05:55 INFO src.financialqa.ingestion.ingestionpipeline line 446  Chunked 0 Documents to 0 using a chunk size of 400 


In [11]:
for i, chunk in enumerate(text_doc_chunks):
# for i, chunk in enumerate(table_doc_chunks):
# for i, chunk in enumerate(chart_doc_chunks):
    metadata = chunk.metadata
    print(f"Page {metadata.get('page_num')} content:", chunk.page_content)
    print('Company name:', metadata.get('company_name'))
    print('Report quarter:', metadata.get('report_quarter'))
    print('Page titles:', metadata.get('page_titles'))
    print('Page headers:', metadata.get('page_headers'))
    print('Section headers:', metadata.get('section_headers'))
    print('Page footers:', metadata.get('page_footers'))
    print('')
    if i == 4:
        break

Page 1 content: CIBCO CIBC CAPITAL MARKETS EQUITY RESEARCH April 26, 2023 Earnings Revision Market Trends Favourable; IFRS 17 Still Presents Many Unknowns Lifecos have outperformed other financials by a good margin year to date. We are somewhat cautious heading into Q1 given the transition to IFRS 17 and lower-than-normal earnings visibility. However, we remain positive on the 12- to 24-month outlook versus banks based on expected EPS growth and capital optionality. The lifecos are trading at 1.3x P/BV on average (IFRS 17 adjusted), in line with the five-year trailing average of 1.3x. Our two Outperformer-rated names are IAG and GWO as we expect both to post strong earnings growth this year. Our investment thesis for the sector is summarized as follows: 1. IFRS 17 reduces near-term earnings visibility. Forecasting 2023 financials and providing a full scope analysis of Q1 results is significantly limited given that 2022 results pro forma IFRS 17 have not been provided by any lifeco. We 

In [16]:
ai_search_index = ingestion_pipeline.get_search_index(
    upload_docs=text_doc_chunks[:10],
    upload_docs_in_batches=True,
    batch_size=5,
    overwrite_index=True,
)

21:07:02 INFO src.financialqa.ingestion.ingestionpipeline line 470  Getting index 'messiah_search' from AI Search service 'nlp-ai-search1'..


21:07:02 INFO src.financialqa.ingestion.ingestionpipeline line 486  Found existing index 'messiah_search' in search service 'nlp-ai-search1'
21:07:02 INFO src.financialqa.ingestion.ingestionpipeline line 593  Overwriting index 'messiah_search'
21:07:04 INFO src.financialqa.ingestion.ingestionpipeline line 620  Attempting to upload batch of 5 Documents to index 'messiah_search'..
21:07:06 INFO src.financialqa.ingestion.ingestionpipeline line 633  Successfully uploaded batch of 5 Documents to index 'messiah_search' in 2 seconds
21:07:06 INFO src.financialqa.ingestion.ingestionpipeline line 620  Attempting to upload batch of 5 Documents to index 'messiah_search'..
21:07:07 INFO src.financialqa.ingestion.ingestionpipeline line 633  Successfully uploaded batch of 5 Documents to index 'messiah_search' in 2 seconds
21:07:07 INFO src.financialqa.ingestion.ingestionpipeline line 639  Successfully uploaded a total of 10 Documents to index 'messiah_search' in 3 seconds


In [20]:
inferencepipeline = InferencePipeline()

21:20:00 INFO src.financialqa.inference.inferencepipeline line 218  Loaded API variables
21:20:00 INFO src.financialqa.inference.inferencepipeline line 144  Getting AzureChatOpenAI completion model 'gpt-4o-2024-05-13-standard-custom' from endpoint https://jhi-apim.prod.manulife.com/internal/technologystrategymgt/usdoltcaoaiprod/v1/usa..
21:20:00 INFO src.financialqa.inference.inferencepipeline line 168  Getting OpenAI embedding model 'text-embedding-ada-002' from endpoint https://jhi-apim.prod.manulife.com/internal/technologystrategymgt/usdoltcaoaiprod/v1/usa..
21:20:00 INFO src.financialqa.inference.inferencepipeline line 188  Getting Azure AI Search index 'messiah_search' from service 'nlp-ai-search1'..


In [18]:
inferencepipeline.invoke_model(
    query='What is the 5-Yr Average P/E?',
    company_name='CIBC',
    top_k=3,
)

21:07:12 INFO src.financialqa.inference.inferencepipeline line 126  Retrieved context: [Document(metadata={'id': 'OGZkZjI3N2YtNTBhNS00YzUwLTk5MWMtOGQxY2I1NmY2MWE4', 'page_num': 2, 'pdf_name': 'CIBC_Preview_Q1', 'company_name': 'CIBC', 'report_quarter': 'Q1', 'page_titles': 'Industry Outlook', 'page_headers': '', 'section_headers': '', 'page_footers': 'This document is intended for Department MAILBOX (Manulife)'}, page_content='The lifecos have delivered strong total returns year to date (7.7% on average) versus the banks (2.1% on average) and the diversified financials covered by CIBC (2.9% on average). Lifeco valuations are no longer discounted, but we believe earnings growth and capital optionality favour the lifecos over banks. We are somewhat cautious heading into Q1 reporting due to the transition to IFRS 17 accounting and what we think are relatively high consensus expectations. However, we have a positive view on the sector for the next 12 to 24 months based on EPS growth, exces